In [1]:
import ee
import geemap
import random
import os
import json

import matplotlib.pyplot as plt
    
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library. Replace with your relevant project title
ee.Initialize(project = os.environ.get("PROJECT_NAME"))

import ee_export_utils # library of functions
import ee_utils

random.seed(123)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
def _get_all_image_collections(viirs_info = None):
    """Gets all the image collections and corresponding time sampling."""
    image_collections = {
      'drought':
          ee_utils.get_image_collection(ee_utils.DataType.DROUGHT_GRIDMET),
      'vegetation':
          ee_utils.get_image_collection(ee_utils.DataType.VEGETATION_VIIRS),
      'weather_gridmet':
          ee_utils.get_image_collection(ee_utils.DataType.WEATHER_GRIDMET),
      'weather_rtma':
          ee_utils.get_image_collection(ee_utils.DataType.WEATHER_RTMA),
      # 'modis_fire':
      #     ee_utils.get_image_collection(ee_utils.DataType.FIRE_MODIS),
      'viirs_fire':
          ee_utils.get_image_collection(ee_utils.DataType.FIRE_VIIRS),
    }
    time_sampling = {
      'drought':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.DROUGHT_GRIDMET],
      'vegetation':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.VEGETATION_VIIRS],
      'weather_gridmet':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.WEATHER_GRIDMET],
      'weather_rtma':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.WEATHER_RTMA],
      # 'modis_fire':
      #     ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.FIRE_MODIS],
      'viirs_fire':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.FIRE_VIIRS],
    }
    return image_collections, time_sampling

def _verify_feature_collection(
    feature_collection
    ):
    """Verifies the feature collection is valid.
    
    If the feature collection is invalid, resets the feature collection.
    
    Args:
    feature_collection: An EE feature collection.
    
    Returns:
    `(feature_collection, size)` a tuple of the verified feature collection and
    its size.
    """
    try:
        size = int(feature_collection.size().getInfo())
        
        # print("Verified a feature collection of size",size) # debugging
        
    except ee.EEException:
        # print('EE exception thrown, resetting') # debugging
        
        # Reset the feature collection
        feature_collection = ee.FeatureCollection([])
        size = 0
    
    return feature_collection, size

#### Get time slices and create custom aggregated features

In [4]:
def _get_time_slices(
    window_start,
    window = 1,
    projection = ee.Projection('EPSG:4326'),  # Defer calling until called by test code
    resampling_scale = 500, # units: meters
    lag = 0,
):
    """Extracts the time slice features.
    
    Args:
    window_start: Start of the time window over which to extract data.
    window: Length of the window (in days).
    projection: projection to reproject all data into.
    resampling_scale: length scale to resample data to.
    lag: Number of days before the fire to extract the features.
    
    Returns:
    A list of the extracted EE images.
    """
    projection = projection.atScale(resampling_scale) # reprojection flow is a bit confusing
    
    window_end = window_start.advance(window, 'day')
    viirs_info = (ee_utils.eeDate_to_string(window_start),ee_utils.eeDate_to_string(window_end)) 

    image_collections, time_sampling = _get_all_image_collections(viirs_info = viirs_info)

    # aggregate out-of-the-box features
    drought = image_collections['drought'].filterDate(
      window_start.advance(-lag - time_sampling['drought'], 'day'),
      window_start.advance(-lag, 'day')).median().reproject(
        projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.DROUGHT_GRIDMET])
            ).resample('bicubic')
    
    vegetation = image_collections['vegetation'].filterDate(
      window_start.advance(-lag - time_sampling['vegetation'], 'day'),
      window_start.advance(
          -lag, 'day')).median().unmask()#.reproject(projection).resample('bicubic')
    
    weather_gridmet = image_collections['weather_gridmet'].filterDate(
      window_start.advance(-lag - time_sampling['weather_gridmet'], 'day'),
      window_start.advance(-lag, 'day')).median().reproject(
          projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_GRIDMET])
            ).resample('bicubic')

    # Create bespoke weather features from hourly data
    avg_humidity = image_collections['weather_rtma'].filterDate(
      window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
      window_start.advance(-lag, 'day')).select('SPFH').mean().reproject(
          projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
            ).resample('bicubic').rename('avg_sph')

    daytime_temp = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select('TMP') \
        .filter(ee.Filter.calendarRange(16, 4, 'hour')) \
        .median() \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('tmp_day')

    q75_temp = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select('TMP') \
        .reduce(ee.Reducer.percentile([75])) \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('tmp_75')

    median_gust = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select('GUST') \
        .median() \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('gust_med')

    avg_wind = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select('WIND') \
        .mean() \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('wind_avg')

    q75_wind = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select('WIND') \
        .reduce(ee.Reducer.percentile([75])) \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('wind_75')

    weighted_wind_dir = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select(['UGRD', 'VGRD', 'WIND']) \
        .map(lambda img: img.select('UGRD').addBands(img.select('WIND'))) \
        .reduce(ee.Reducer.mean().splitWeights()) \
        .addBands(
            image_collections['weather_rtma'].filterDate(
                window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
                window_start.advance(-lag, 'day')) \
            .select(['UGRD', 'VGRD', 'WIND']) \
            .map(lambda img: img.select('VGRD').addBands(img.select('WIND'))) \
            .reduce(ee.Reducer.mean().splitWeights())
        ) \
        .expression('atan2(b("mean_1"), b("mean"))') \
        .select([0]) \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('wdir_wind')

    weighted_gust_dir = image_collections['weather_rtma'].filterDate(
        window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
        window_start.advance(-lag, 'day')) \
        .select(['UGRD', 'VGRD', 'GUST']) \
        .map(lambda img: img.select('UGRD').addBands(img.select('GUST'))) \
        .reduce(ee.Reducer.mean().splitWeights()) \
        .addBands(
            image_collections['weather_rtma'].filterDate(
                window_start.advance(-lag - time_sampling['weather_rtma'], 'day'),
                window_start.advance(-lag, 'day')) \
            .select(['UGRD', 'VGRD', 'GUST']) \
            .map(lambda img: img.select('VGRD').addBands(img.select('GUST'))) \
            .reduce(ee.Reducer.mean().splitWeights())
        ) \
        .expression('atan2(b("mean_1"), b("mean"))') \
        .select([0]) \
        .reproject(
            projection.atScale(ee_utils.RESAMPLING_SCALE[ee_utils.DataType.WEATHER_RTMA])
        ).resample('bicubic').rename('wdir_gust')
    
    custom_weather_features = [
        avg_humidity,
        daytime_temp,
        q75_temp,
        median_gust,
        avg_wind,
        q75_wind,
        weighted_wind_dir,
        weighted_gust_dir
    ]

    # modis_prev_fire = image_collections['modis_fire'].filterDate(
    #   window_start.advance(-lag - time_sampling['modis_fire'], 'day'),
    #   window_start.advance(-lag, 'day')).map(
    #       ee_utils.remove_mask).max().rename('modis_PrevFireMask')

    viirs_prev_fire = image_collections['viirs_fire'].filterDate(
      window_start.advance(-lag - time_sampling['viirs_fire'], 'day'),
      window_start.advance(-lag, 'day')).map(
          ee_utils.remove_mask).max().rename('viirs_PrevFireMask')
    
    # modis_fire = image_collections['modis_fire'].filterDate(window_start, window_end).map(
    #   ee_utils.remove_mask).max().rename('modis_FireMask')

    viirs_fire = image_collections['viirs_fire'].filterDate(window_start,window_end).map(
      ee_utils.remove_mask).max().rename('viirs_FireMask')
    
    detection1 = viirs_prev_fire.rename('detection') # can use the MODIS data for detection, but have to subtract 6 from all measurements
    detection2 = viirs_fire.rename('detection')
    
    final_slices = [vegetation, drought, weather_gridmet] + \
      custom_weather_features + \
    [viirs_prev_fire, viirs_fire, detection1, detection2]
    
    return final_slices

### Define parameters and begin data export

In [2]:
config = {
    'bucket':'modified_ndws',
    'folder':'ndws_west_conus_dataset',
    'start_date':ee.Date('2015-01-01'),
    'end_date':ee.Date('2023-12-31'),
    'prefix':'ndws_west',
    'kernel_size':64, # size of export images in pixels
    'sampling_scale':500, # size of pixel in meters
    'eval_split_ratio':0.2,
    'num_samples_per_file':1000,
    'coordinates':ee_utils.COORDINATES['US_WEST']
}

In [3]:
with open('ndws_conus_west.json','w') as json_file:
    json.dump({x:str(config[x]) for x in config},json_file)

In [4]:
res = ee_export_utils.export_ml_datasets(
      bucket=config['bucket'],
      folder=config['folder'],
      start_date=config['start_date'],
      end_date=config['end_date'],
      prefix=config['prefix'],
      kernel_size=config['kernel_size'],
      sampling_scale=config['sampling_scale'],
      eval_split_ratio=config['eval_split_ratio'],
      num_samples_per_file=config['num_samples_per_file'],
      coordinates=config['coordinates'],
      fire_season_only = True
  )

Beginning export process in mode train
Features to be extracted:
['elevation', 'chili', 'impervious', 'water', 'population', 'fuel1', 'fuel2', 'fuel3', 'landfire_fuel', 'NDVI', 'pdsi', 'pr', 'erc', 'bi', 'avg_sph', 'tmp_day', 'tmp_75', 'gust_med', 'wind_avg', 'wind_75', 'wdir_wind', 'wdir_gust', 'viirs_PrevFireMask', 'viirs_FireMask']
Looking at date 2016-07-22T00:00:00
Getting detection count....
Prev fire count found: 1
Post fire count found: 1
Extracting samples in chunks...


EEException: Reprojection output too large (13595x14663 pixels).

In [6]:
res.size()

/Applications/miniconda3/envs/fire/lib/python3.12/site-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Reprojection output too large (20384x21732 pixels).'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [12]:
# Create a map centered on the CONUS
Map = geemap.Map(center=[40, -96], zoom=4)

# Get the first image from the collection
# idx =22

# print(feature_names[idx])
# img = res[-1]

# Add the first image to the map
viz_params = {
    'min': 0,
    'max': 1,
    'palette': ['black','red'],
    'opacity':0.7
}

Map.addLayer(res, viz_params, )#feature_names[idx])
Map

KeyboardInterrupt: 